In [119]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from h3 import h3
import json
from urllib.request import URLError, Request, urlopen

In [120]:
df = pd.read_csv('Data/LaGuardia_Dropoff_Final.csv')
start_date = '2016-01-29 08:00:00' # Start date with time
end_date = '2016-01-30 23:59:59' # End date with time
df=df[(df['tpep_pickup_datetime'] >= start_date) & (df['tpep_pickup_datetime'] <= end_date)]

#select required columns
df=df[['tpep_pickup_datetime','tpep_dropoff_datetime', 'passenger_count', 'trip_distance','pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude']]
#get h3 indices
df['pickup_h3'] = df.apply(lambda x: h3.geo_to_h3(x['pickup_latitude'], x['pickup_longitude'], 8), axis=1)
df['dropoff_h3'] = df.apply(lambda x: h3.geo_to_h3(x['dropoff_latitude'], x['dropoff_longitude'], 10), axis=1)

# Installation steps


### 1.Install the latest JRE and get GraphHopper Server as zip from <a href=https://graphhopper.com/public/releases/graphhopper-web-0.10.3-bin.zip>Graphhopper API</a>. Unzip it.
### 2.Copy this OSM file into the SAME unzipped directory: <a href=https://download.geofabrik.de/north-america/us/new-york-latest.osm.pbf >new-york-latest.osm.pbf</a>
### 3.Start GraphHopper Maps via: ava -jar graphhopper-web-0.10.3-with-dep.jar jetty.resourcebase=webapp config=config-example.properties datareader.file=new-york-latest.osm.pbf. 
### 3.Test to see if its running after you see 'Started server at HTTP 8989' by going to http://localhost:8989/ and you should see a map of New York.
### 4.Keep this running when executing our program because this is the API

In [124]:
#Check how graphhopper works
a,b,c,d = df['pickup_latitude'][0],df['pickup_longitude'][0],df['dropoff_latitude'][0],df['dropoff_longitude'][0]
request_str = 'http://localhost:8989/route?point=' + str(a) + '%2C' + str(b) + '&point=' + str(
            c) + '%2C' + str(d) + '&vehicle=car'
request = Request(request_str)
res=requests.get(request_str)
print("Distance = {}".format(json.loads(res.text)['paths'][0]['distance']))
print("Time = {}".format(json.loads(res.text)['paths'][0]['time']))

Distance = 14964.969
Time = 892052
